In [1]:
raw = '''Valve DR has flow rate=22; tunnels lead to valves DC, YA
Valve IO has flow rate=14; tunnels lead to valves GE, CK, HY, XB
Valve XY has flow rate=0; tunnels lead to valves IP, AR
Valve UQ has flow rate=0; tunnels lead to valves XU, PD
Valve FO has flow rate=0; tunnels lead to valves DL, NC
Valve PU has flow rate=0; tunnels lead to valves ZJ, AN
Valve MK has flow rate=0; tunnels lead to valves ZS, SB
Valve HN has flow rate=0; tunnels lead to valves AA, DV
Valve XF has flow rate=0; tunnels lead to valves XB, AA
Valve OD has flow rate=13; tunnels lead to valves ZS, AF, SY, QQ, AR
Valve GE has flow rate=0; tunnels lead to valves KR, IO
Valve UF has flow rate=18; tunnels lead to valves QQ, AN, YE, GY
Valve WK has flow rate=19; tunnel leads to valve PQ
Valve PQ has flow rate=0; tunnels lead to valves WK, CW
Valve XU has flow rate=0; tunnels lead to valves DV, UQ
Valve SH has flow rate=0; tunnels lead to valves IP, AA
Valve SY has flow rate=0; tunnels lead to valves ZJ, OD
Valve OU has flow rate=0; tunnels lead to valves CK, DL
Valve IP has flow rate=8; tunnels lead to valves CY, ML, YI, XY, SH
Valve XZ has flow rate=0; tunnels lead to valves AM, PD
Valve ZU has flow rate=0; tunnels lead to valves CW, SB
Valve DC has flow rate=0; tunnels lead to valves CF, DR
Valve QY has flow rate=0; tunnels lead to valves CW, MQ
Valve XB has flow rate=0; tunnels lead to valves IO, XF
Valve AF has flow rate=0; tunnels lead to valves PD, OD
Valve GY has flow rate=0; tunnels lead to valves UF, ZC
Valve ZC has flow rate=0; tunnels lead to valves GY, CW
Valve ZJ has flow rate=25; tunnels lead to valves SY, PU
Valve NC has flow rate=6; tunnels lead to valves HY, ML, NJ, AT, FO
Valve DS has flow rate=0; tunnels lead to valves AT, DV
Valve DV has flow rate=7; tunnels lead to valves FD, KR, HN, DS, XU
Valve HY has flow rate=0; tunnels lead to valves NC, IO
Valve WF has flow rate=0; tunnels lead to valves NJ, AA
Valve CK has flow rate=0; tunnels lead to valves IO, OU
Valve YE has flow rate=0; tunnels lead to valves CY, UF
Valve LA has flow rate=0; tunnels lead to valves DL, ZM
Valve QQ has flow rate=0; tunnels lead to valves OD, UF
Valve AM has flow rate=0; tunnels lead to valves XZ, SB
Valve AN has flow rate=0; tunnels lead to valves UF, PU
Valve CL has flow rate=16; tunnels lead to valves YA, LD
Valve CF has flow rate=12; tunnel leads to valve DC
Valve FD has flow rate=0; tunnels lead to valves DV, DL
Valve QU has flow rate=0; tunnels lead to valves LD, PD
Valve AT has flow rate=0; tunnels lead to valves DS, NC
Valve SB has flow rate=24; tunnels lead to valves MK, AM, ZU
Valve YI has flow rate=0; tunnels lead to valves DL, IP
Valve ZM has flow rate=0; tunnels lead to valves AA, LA
Valve LD has flow rate=0; tunnels lead to valves CL, QU
Valve AR has flow rate=0; tunnels lead to valves OD, XY
Valve DL has flow rate=5; tunnels lead to valves FO, LA, YI, OU, FD
Valve MQ has flow rate=0; tunnels lead to valves QY, PD
Valve PD has flow rate=9; tunnels lead to valves MQ, QU, XZ, AF, UQ
Valve KR has flow rate=0; tunnels lead to valves GE, DV
Valve CY has flow rate=0; tunnels lead to valves YE, IP
Valve AA has flow rate=0; tunnels lead to valves SH, XF, ZM, HN, WF
Valve NJ has flow rate=0; tunnels lead to valves NC, WF
Valve YA has flow rate=0; tunnels lead to valves CL, DR
Valve ML has flow rate=0; tunnels lead to valves NC, IP
Valve CW has flow rate=15; tunnels lead to valves QY, PQ, ZC, ZU
Valve ZS has flow rate=0; tunnels lead to valves MK, OD'''

In [2]:
lines = raw.splitlines()

In [3]:
import re
p = re.compile(r'Valve (.*) has flow rate=(\d+); tunnels? leads? to valves? (.*)')

In [4]:
vs = {}
for line in lines:
  v, flow, neighbors= p.match(line).groups()
  vs[v] = (int(flow), neighbors.split(', '))

In [5]:
nz_vs = {v for v, (f, n) in vs.items() if f > 0}

In [6]:
import collections
s = nz_vs | {'AA'}
dists = {(v, v): 0 for v in s}
for v in s:
  visited = set()
  q = collections.deque([v])
  while q:
    c = q.popleft()
    for n in vs[c][1]:
      if n in visited:
        continue
      visited.add(n)
      dists[v, n] = dists[v, c] + 1
      q.append(n)

In [7]:
stack = [('AA', [], 30, 0)]
max_score = 0
while stack:
  curr, visited, remaining, score = stack.pop()
  for v in nz_vs - set(visited):
    new_remaining = remaining - dists[curr, v] - 1
    if new_remaining <= 0:
      continue
    new_score = score + new_remaining * vs[v][0]

    max_score = max(max_score, new_score)
    stack.append((v, visited + [v], new_remaining, new_score))
max_score

1850

In [8]:
stack = [('AA', [], 26, 0)]
max_scores = {(): 0}
while stack:
  curr, visited, remaining, score = stack.pop()
  visited = list(visited)
  for v in nz_vs - set(visited):
    new_remaining = remaining - dists[curr, v] - 1
    if new_remaining <= 0:
      continue
    new_visited = tuple(sorted(visited + [v]))
    new_score = score + new_remaining * vs[v][0]

    max_scores[new_visited] = max(max_scores.get(new_visited, 0), new_score)
    stack.append((v, new_visited, new_remaining, new_score))

In [9]:
from itertools import chain, combinations
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [10]:
s_nz_vs = list(sorted(nz_vs))

In [11]:
ordered_scores = sorted(max_scores.items(), key=lambda kv: kv[1], reverse=True)
for s in powerset(s_nz_vs):
  if s in max_scores:
    continue
  max_scores[s] = max(max_scores[tuple(x for x in s if x != v)] for v in s)

In [12]:
m_score = 0
for s in powerset(s_nz_vs):
  m_score = max(m_score, max_scores[s] + max_scores[tuple(v for v in s_nz_vs if v not in s)])
m_score

2306